# Computations
## Data import

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pandas as pd
import numpy as np
import re
import scipy
import itertools
import pickle
import time
import os.path
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.cm import ScalarMappable, get_cmap
from matplotlib.colors import Normalize
import seaborn as sns
import statsmodels.api as sm
from statsmodels.distributions.empirical_distribution import ECDF as sm_ECDF
import statsmodels.formula.api as smf
from statsmodels.genmod.bayes_mixed_glm import BinomialBayesMixedGLM
import concurrent.futures
import attila_utils
import gem_tools

In [2]:
groupdict = {
    #'m-control': ('all_control', 'MSBB'),
    'm-AD-A': ('SubtypeA_AD', 'MSBB'),
    'm-AD-B1': ('SubtypeB1_AD', 'MSBB'),
    'm-AD-B2': ('SubtypeB2_AD', 'MSBB'),
    'm-AD-C1': ('SubtypeC1_AD', 'MSBB'),
    'm-AD-C2': ('SubtypeC2_AD', 'MSBB'),
}
ar = gem_tools.read_active_reactions(groupdict=groupdict)

In [3]:
vcp_p = 0.2
fe_p = 2
fit_method = 'fit_vb'
gemsubsys = gem_tools.read_gem_excel()['SUBSYSTEM']
subsys = gemsubsys.unique()[2]
data = gem_tools.long_ar_subsys([subsys], ar=ar, gemsubsys=gemsubsys)
random = {'Subtypes': 'disease_state', 'Reactions': 'rxn_ID', 'Subjects': 'subject_ID'}
formula = 'rxn_state ~ 1'
md = BinomialBayesMixedGLM.from_formula(formula, random, data, vcp_p=vcp_p, fe_p=fe_p)
fit = getattr(md, fit_method)
m = fit()
m.summary()

,Type,Post. Mean,Post. SD,SD,SD (LB),SD (UB)
Intercept,M,-0.5524,0.0664,,,
Subtypes,V,-0.1345,0.1881,0.874,0.600,1.273
Reactions,V,1.4838,0.0842,4.410,3.727,5.218
Subjects,V,-0.3125,0.0567,0.732,0.653,0.819


In [4]:
data

rxn_state disease_state    rxn_ID  \
ID                                                                  
MAR03944 P19B648.BM_36_517              1        m-AD-A  MAR03944   
         S110B355.BM_36_308             1        m-AD-A  MAR03944   
         P19B648.BM_36_518              1        m-AD-A  MAR03944   
         P19B648.BM_36_545              1        m-AD-A  MAR03944   
         S111B355.BM_36_398             1        m-AD-A  MAR03944   
...                                   ...           ...       ...   
MAR01472 S111B355.BM_36_384             0       m-AD-C2  MAR01472   
         E007C014.hB_RNA_10567          0       m-AD-C2  MAR01472   
         S151B648.BM_36_495             0       m-AD-C2  MAR01472   
         S111B355.BM_36_425             0       m-AD-C2  MAR01472   
         E007C014.hB_RNA_10632          0       m-AD-C2  MAR01472   

                                           subject_ID  
ID                                                     
MAR03944 P19B648.BM_36_517          P19B648.BM_36_517  
         S110B355.BM_36_308        S110B355.BM_36_308  
         P19B648.BM_36_518          P19B648.BM_36_518  
         P19B648.BM_36_545          P19B648.BM_36_545  
         S111B355.BM_36_398        S111B355.BM_36_398  
...                                               ...  
MAR01472 S111B355.BM_36_384        S111B355.BM_36_384  
         E007C014.hB_RNA_10567  E007C014.hB_RNA_10567  
         S151B648.BM_36_495        S151B648.BM_36_495  
         S111B355.BM_36_425        S111B355.BM_36_425  
         E007C014.hB_RNA_10632  E007C014.hB_RNA_10632  

[3171 rows x 4 columns]

## Modeling

$y \sim \beta_0 + b_1 z_1 + b_2 z_2$

In [5]:
random={'Subtypes': 'disease_state', 'Reactions': 'rxn_ID'}
subsys = gemsubsys.unique()[2]
print(subsys)
m = gem_tools.myBinomialBayesRandomGLM([subsys], random=random, ar=ar, vcp_p=0.2, fe_p=2, fit_method='fit_map')
m.summary()

Galactose metabolism


,Type,Post. Mean,Post. SD,SD,SD (LB),SD (UB)
Intercept,M,-0.6494,1.7671,,,
Subtypes,V,-0.1484,0.1919,0.862,0.587,1.265
Reactions,V,1.2927,0.1038,3.643,2.960,4.483


In [6]:
random={'Subtypes': 'disease_state'}
subsys = gemsubsys.unique()[2]
print(subsys)
m = gem_tools.myBinomialBayesRandomGLM([subsys], random=random, ar=ar, vcp_p=0.2, fe_p=2, fit_method='fit_map')
m.summary()

Galactose metabolism


,Type,Post. Mean,Post. SD,SD,SD (LB),SD (UB)
Intercept,M,0.3072,0.7637,,,
Subtypes,V,-0.1937,0.1989,0.824,0.554,1.226


In [7]:
fpath = '../../results/2025-05-25-ad-subtype-random-fx/fitted_models.pickle'
if os.path.exists(fpath):
    print('exists')
    with open(fpath, 'rb') as f:
        fitted_models = pickle.load(f)
else:
    subsystems = gemsubsys.unique()
    random = {'Subtypes': 'disease_state', 'Reactions': 'rxn_ID'}
    d = {subsys: gem_tools.myBinomialBayesRandomGLM([subsys], random=random, ar=ar, vcp_p=0.2, fe_p=2, fit_method='fit_map') for subsys in subsystems}
    fitted_models = pd.Series(d)
    with open(fpath, 'wb') as f:
        pickle.dump(fitted_models, f)

exists


# Results

In [8]:
ix = pd.MultiIndex.from_product([['Subtypes', 'Reactions'], ['mean', 'stdev']])
vcp_estim = fitted_models.dropna().apply(lambda m: pd.Series([m.vcp_mean[0], m.vcp_sd[0], m.vcp_mean[1], m.vcp_sd[1]], index=ix))
vcp_estim = vcp_estim.sort_values(('Subtypes', 'mean'), ascending=False)
vcp_estim

Subtypes            \
                                                        mean     stdev   
Acyl-CoA hydrolysis                                 0.433030  0.141009   
Chondroitin sulfate degradation                     0.196213  0.151013   
Beta oxidation of branched-chain fatty acids (m...  0.158158  0.177318   
Keratan sulfate degradation                         0.145885  0.159221   
Carnitine shuttle (mitochondrial)                   0.069403  0.165362   
...                                                      ...       ...   
Fatty acid oxidation                               -0.198412  0.199741   
Eicosanoid metabolism                              -0.198526  0.199883   
Beta oxidation of even-chain fatty acids (perox... -0.198970  0.199836   
Lipoic acid metabolism                             -0.199007  0.200177   
Artificial reactions                               -0.199121  0.200158   

                                                   Reactions            
                                                        mean     stdev  
Acyl-CoA hydrolysis                                 0.578702  0.084623  
Chondroitin sulfate degradation                     0.902637  0.091334  
Beta oxidation of branched-chain fatty acids (m... -0.347457  0.211399  
Keratan sulfate degradation                         1.112690  0.074233  
Carnitine shuttle (mitochondrial)                   0.445418  0.074815  
...                                                      ...       ...  
Fatty acid oxidation                                1.637815  0.034006  
Eicosanoid metabolism                               1.435973  0.077497  
Beta oxidation of even-chain fatty acids (perox...  1.607062  0.082999  
Lipoic acid metabolism                              1.309296  0.114957  
Artificial reactions                                1.388042  0.102484  

[128 rows x 4 columns]

In [9]:
data = gem_tools.long_ar_subsys(['Acyl-CoA hydrolysis'], ar=ar, gemsubsys=gemsubsys)
data

rxn_state disease_state    rxn_ID  \
ID                                                                  
MAR00154 P19B648.BM_36_517              1        m-AD-A  MAR00154   
         S110B355.BM_36_308             1        m-AD-A  MAR00154   
         P19B648.BM_36_518              1        m-AD-A  MAR00154   
         P19B648.BM_36_545              1        m-AD-A  MAR00154   
         S111B355.BM_36_398             1        m-AD-A  MAR00154   
...                                   ...           ...       ...   
MAR03477 S111B355.BM_36_384             1       m-AD-C2  MAR03477   
         E007C014.hB_RNA_10567          1       m-AD-C2  MAR03477   
         S151B648.BM_36_495             1       m-AD-C2  MAR03477   
         S111B355.BM_36_425             1       m-AD-C2  MAR03477   
         E007C014.hB_RNA_10632          1       m-AD-C2  MAR03477   

                                           subject_ID  
ID                                                     
MAR00154 P19B648.BM_36_517          P19B648.BM_36_517  
         S110B355.BM_36_308        S110B355.BM_36_308  
         P19B648.BM_36_518          P19B648.BM_36_518  
         P19B648.BM_36_545          P19B648.BM_36_545  
         S111B355.BM_36_398        S111B355.BM_36_398  
...                                               ...  
MAR03477 S111B355.BM_36_384        S111B355.BM_36_384  
         E007C014.hB_RNA_10567  E007C014.hB_RNA_10567  
         S151B648.BM_36_495        S151B648.BM_36_495  
         S111B355.BM_36_425        S111B355.BM_36_425  
         E007C014.hB_RNA_10632  E007C014.hB_RNA_10632  

[9664 rows x 4 columns]

In [10]:
def get_frac_active_rxn(subsys):
    data = gem_tools.long_ar_subsys([subsys], ar=ar, gemsubsys=gemsubsys)
    s = data.groupby('disease_state').apply(lambda df: df.rxn_state.sum() / df.shape[0])
    df = s.to_frame(subsys).transpose()
    return df

l = [get_frac_active_rxn(subsys) for subsys in vcp_estim.index]

In [ ]:
frac_active_rxn = pd.concat(l)
frac_active_rxn.columns = pd.MultiIndex.from_product([['Fraction of active reactions'], frac_active_rxn.columns])
entropy = frac_active_rxn.apply(scipy.stats.entropy, axis=1).to_frame(('Entropy', ''))

results = pd.concat([vcp_estim, frac_active_rxn, entropy], axis=1)
results.to_excel('Sheet1', '../../results/2025-05-25-ad-subtype-random-fx/AD-subtype-variance.xlsx')
results

In [ ]:
results.sort_values(('Entropy', ''), ascending=True)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x=results[('Subtypes', 'mean')], y=results[('Entropy', '')])
ax.set_xlabel('Subtypes variance posterior mean')
ax.set_ylabel('Entropy')

In [ ]:
%connect_info